# Almacenando datos de Twitter en Riak

![png](https://upload.wikimedia.org/wikipedia/en/8/8e/Riak_distributed_NoSQL_key-value_data_store_logo.png)

Se van a crear los siguientes buckets:

|bucket|Clave|Contenido|
|-|-|-|
|tweets|Id del tweet|Datos del Tweet en formnato JSON. Está incluido el tweet retwiteado y los datos del usuario|
|users|Código de usuario (screeen_name)|Datos del usurario en formato JSON|
|hashtag|Hashtag|Nº de veces que aparce el hashtag en cualquiera de los tweets|

El bucket de tweets tendrá los siguientes índices:

|Nombre del índice|Contenido|¿Que busquedas permite hacer?|
|-|-|-|
|idx_usr_bin|Usuario que crea el tweet|Tweets que ha creado un determinado usuario|
|idx_hashtag_bin|Hashtag que se menciona en el tweet|Tweets donde aparece un determinado hashtag|
|idx_user_mentioned_bin|Usuario que se mencionan en el tweet| Tweets donde se menciona a un determinado usuario|

Una representación gráfica de Riak sería la siguiente

<p><p><p>
<img src="./images/riak_twitter.png" width="550" height="100">

In [ ]:
import riak
from pprintpp import pprint as pp
import json

In [ ]:
# connect to database
myClient = riak.RiakClient()
myClient.ping()

In [ ]:
BUCKET_TWEETS = 'tweets'
BUCKET_USERS = 'users'
BUCKET_HASHTAGS = 'hashtags'

tweets = myClient.bucket(BUCKET_TWEETS)
users = myClient.bucket(BUCKET_USERS)
hashtags = myClient.bucket(BUCKET_HASHTAGS)


In [ ]:
def insert_hashtag(hashtag):
    count = hashtags.get(hashtag).data
    if count == None:
        count = 1
    else:
        count = count + 1

    hashtags.new(hashtag, data = count).store()

In [ ]:
def insert_user(user_json, replace = True):
    key_user = user_json["screen_name"]
    if replace or not users.get(key_user).exists:
        user = users.new(key_user, user_json)
        user.store()
        
    return key_user
    

In [ ]:
def insert_tweet(tweet_json):
    tweet = tweets.new(tweet_json["id_str"], tweet_json)
    user = insert_user(tweet_json['user'])
    tweet.add_index('idx_usr_bin', user)
    
    for hashtag in tweet_json['entities']:
        tweet.add_index('idx_hashtag_bin', hashtag)   
        insert_hashtag(hashtag)
        
    for user_mentioned in tweet_json['user_mentions']:
        tweet.add_index('idx_user_mentioned_bin', user_mentioned["screen_name"])   
        insert_user(user_mentioned, replace = False)    
        
    tweet.store()     

    if 'retweeted_status' in tweet_json:
        insert_tweet(tweet_json['retweeted_status'])


In [ ]:
tweets_data_path = '../../data/tweets.json'
tweets_file = open(tweets_data_path, "r")

i = 1
for line in tweets_file:
    tweet_json = json.loads(line)
    
    try:
        insert_tweet(tweet_json)
        i = i + 1
    except:
        pass

        
print("%s tweets procesados" % i)

## Imprimir el contenido de 1 Tweet

Imprimir el contenido del tweet con el id 655039580312371200

In [ ]:
tweet = tweets.get("655039580312371200").data

pp(tweet)

## Mostrar la información asociada a 1 usuario

Mostrar la información asociada al usuario DBaker007

In [ ]:
user = users.get("DBaker007").data

pp(user)

## Listar los Hashtags de mas de 100 tweets

In [ ]:
riak.disable_list_exceptions = True
for keys in hashtags.stream_keys():
    for key in keys:
        count = hashtags.get(key).data
        if count > 100:
            print('Hashtag %s:%s' % (key,  count))

riak.disable_list_exceptions = False

## Listar los tweets asociados a 1 Hashtag

In [ ]:
keys = tweets.stream_index("idx_hashtag_bin", 'NoSQL')
for keys in keys.results:
    for key in keys:
        tweet =  tweets.get(key).data
        
user = tweet["user"]["screen_name"]
print("Último Tweet:")
pp(tweet)
print("User:", user)


## Mostrar los tweets asociados a 1 usuario

In [ ]:
keys = tweets.stream_index("idx_usr_bin", user)
for keys in keys.results:
    for tweet_key in keys:
        tweet = tweets.get(tweet_key).data
        print("Tweet:", tweet_key)
        
pp(tweet)


## Listado de usuarios

In [ ]:
i = 1
riak.disable_list_exceptions = True
for keys in users.stream_keys():  
    for key in keys:
        print('User %s' % key)
        i = i + 1
    
    if i > 10:
        break
        
riak.disable_list_exceptions = False        